## Dependencias

In [40]:
import pandas as pd
import json #JSON = Java Script Object Notation
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR,FLOAT,INTEGER,DATE
import os

pd.set_option('display.max_columns',None)

## Credenciales

In [11]:
creds = json.load(open('credenciales.json','rb'))

## Crear conexión a base de datos

In [13]:
url = f"mysql+pymysql://{creds['user']}:{creds['password']}@{creds['servidor']}/{creds['base_de_datos']}"
cnx = create_engine(url,encoding='utf8')
cnx = cnx.connect()

In [14]:
cnx.closed

False

## Lectura y limpieza de datos

In [16]:
ruta = '/home/jose/Documentos/bd/delitos/'
listaArchivos = sorted([os.path.join(ruta,arch) for arch in os.listdir(ruta)])
arch = listaArchivos[0]

### Función de carga de datos

In [46]:
def cargarDatos(arch:str):
    datos = pd.read_csv(arch,usecols=['entidad','municipio','delito','carpetas','tasa','fecha'])
    datos['fecha'] = pd.to_datetime(datos['fecha'].map(lambda x:x+'-01'),format='%Y-%m-%d')    
    columnas_varchar = ['entidad','municipio','delito']
    campos = dict(zip(columnas_varchar,map(lambda c:VARCHAR(datos[c].map(len).max()),columnas_varchar)))
    campos.update({'carpetas':INTEGER,'tasa':FLOAT,'fecha':DATE})
    datos.to_sql(con=cnx,
             if_exists='append',
             name='historico',
             index=False,
             chunksize=1000,
             dtype=campos)

### Cargar archivos a la nube

In [52]:
# map(cargarDatos,listaArchivos)    
for arch in listaArchivos[-1:]:
    cargarDatos(arch)

In [57]:
datos = pd.read_csv(arch,usecols=['entidad','municipio','delito','carpetas','tasa','fecha'])

In [58]:
datos.entidad.value_counts()

Oaxaca                             9120
Puebla                             3472
Veracruz de Ignacio de la Llave    3392
México                             2000
Jalisco                            2000
Chiapas                            1888
Michoacán de Ocampo                1808
Yucatán                            1696
Hidalgo                            1344
Guerrero                           1296
Sonora                             1152
Chihuahua                          1072
Tlaxcala                            960
San Luis Potosí                     928
Zacatecas                           928
Nuevo León                          816
Guanajuato                          736
Tamaulipas                          688
Durango                             624
Coahuila de Zaragoza                608
Morelos                             528
Nayarit                             320
Querétaro                           288
Sinaloa                             288
Tabasco                             272
